In [ ]:
# === Colab 환경 설정 ===
# 이 셀은 Google Colab에서만 실행됩니다.

import os, sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 Google Colab 환경 감지됨")
    print("필요한 패키지를 설치합니다...")
    
    # 기본 패키지 설치
    !pip install -q numpy matplotlib otter-grader
    
    # 저장소 클론 (이미 있으면 건너뜀)
    if not os.path.exists('/content/toy_BBN'):
        !git clone -q https://github.com/cosmo-ojisan/toy_BBN.git /content/toy_BBN
    
    os.chdir('/content/toy_BBN/dist/student')
    sys.path.insert(0, '/content/toy_BBN/dist/student')
    
    # 한글 폰트 설치 (Colab용)
    !apt-get -qq install -y fonts-nanum > /dev/null 2>&1
    
    print("✅ 환경 설정 완료!")
    print(f"현재 작업 위치: {os.getcwd()}")
else:
    print("💻 로컬 환경 감지됨 - 설정을 건너뜁니다.")

# 4차시: 질량수 7 경로와 리튬-7 문제 (심화)

## 학습 목표

이 노트북을 완료하면 다음을 할 수 있다:

1. **질량수 7** 핵종 ($^7$Li, $^7$Be)의 생성/파괴 경로를 이해한다
2. **12개 핵심 반응**으로 구성된 확장 네트워크를 완성한다
3. **리튬-7 문제**가 무엇인지 설명한다
4. 관측과 이론의 불일치에 대한 **과학적 태도**를 갖는다

---

## 사전 요구사항

- 1-3차시 완료 (동결, 병목, 8반응 네트워크)

In [ ]:
# === 환경 설정 ===
# Google Colab에서 실행할 경우 아래 주석을 해제하세요
# !pip install -q otter-grader

import math
import numpy as np
import matplotlib.pyplot as plt
import warnings

# matplotlib 폰트 경고 억제
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

# 한글 폰트 설정
try:
    import matplotlib.font_manager as fm
    korean_fonts = ['NanumSquare', 'NanumGothic', 'NanumBarunGothic', 
                    'Malgun Gothic', 'AppleGothic', 'UnDotum', 'Noto Sans CJK KR']
    font_found = False
    for font in korean_fonts:
        try:
            fm.findfont(font, fallback_to_default=False)
            plt.rcParams['font.family'] = font
            font_found = True
            break
        except:
            continue
    if not font_found:
        # Colab 환경에서 나눔폰트 설치 시도
        import subprocess
        import sys
        try:
            subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'koreanize-matplotlib'], 
                          capture_output=True, timeout=30)
            import koreanize_matplotlib
        except:
            pass
except:
    pass

plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

print("✅ 환경 설정 완료!")

In [ ]:
# Otter grader 준비
import os, sys

try:
    import otter
except ImportError:
    !pip install -q otter-grader
    import otter

assert os.path.isdir("tests"), "tests 폴더가 없습니다. 저장소를 제대로 클론했는지 확인하세요."
grader = otter.Notebook()

print("✅ 준비 완료: grader.check('qA1') 처럼 즉시채점을 실행할 수 있어요.")

In [ ]:
# Helper: 한국어 채점 출력
import re, os, sys, glob
from io import StringIO
from contextlib import redirect_stdout, redirect_stderr

# 이 레슨의 테스트 접두사
_LESSON_PREFIXES = ['qJ', 'qK', 'qL']

def _list_valid_tests():
    """이 레슨의 테스트 파일 목록 반환"""
    tests = []
    for f in glob.glob('tests/q*.py'):
        name = os.path.splitext(os.path.basename(f))[0]
        if any(name.startswith(prefix) for prefix in _LESSON_PREFIXES):
            tests.append(name)
    return sorted(tests)

def _test_vars_from_file(test_name):
    p = os.path.join('tests', test_name + '.py')
    if not os.path.exists(p):
        return set()
    with open(p, 'r', encoding='utf-8') as f:
        txt = f.read()
    return set(re.findall(r"(ans_[A-Za-z0-9_]+)", txt))

# 원본 grader.check
_original_check = grader.check

def _korean_check(test_name, show_output=True):
    """한국어 메시지와 함께 채점"""
    result = _original_check(test_name)
    if show_output:
        if result.passed:
            print(f"✅ {test_name}: 정답!")
        else:
            needed_vars = _test_vars_from_file(test_name)
            g = globals()
            missing = [v for v in needed_vars if v not in g]
            if missing:
                print(f"❌ {test_name}: 변수 누락 → {missing}")
            else:
                print(f"❌ {test_name}: 오답 — 다시 확인해보세요.")
    return result

grader.check = _korean_check

def _korean_check_all():
    """이 레슨의 테스트만 실행"""
    print("📊 전체 채점 시작...\n")
    valid_tests = _list_valid_tests()
    passed = 0
    failed = 0
    for test_name in valid_tests:
        result = _korean_check(test_name, show_output=False)
        if result.passed:
            passed += 1
        else:
            print(f"❌ {test_name}: 오답")
            failed += 1
    print(f"\n📊 전체 결과: {passed}/{passed+failed} 통과")
    if failed == 0:
        print("🎉 모든 문항 정답!")

grader.check_all = _korean_check_all

---# 1-3차시 핵심 함수 (독립 실행용)4차시를 독립적으로 실행하기 위해 1-3차시의 핵심 함수들을 아래에 포함합니다.**이 셀은 수정하지 마세요!**

In [ ]:
# === 1-3차시 핵심 상수 및 함수 ===# (4차시 독립 실행을 위해 포함)# 물리 상수A_default = 1.32        # t = A / T^2 관계의 상수 [초·MeV^2]gamma0_default = 1.3    
# Gamma = gamma0 * T^5 관계의 상수 [1/초·MeV^5]Q_default = 1.293       # 중성자-양성자 질량차 [MeV]tau_n_default = 879.0   # 중성자 평균 수명 [초]
def t_of_T(T, A=A_default):    """온도 T에서 시간 t 계산 (토이 모델)"""    
    return A / (T * T)
def Gamma_of_T(T, gamma0=gamma0_default):    """온도 T에서 약한 반응률 Gamma 계산"""    
    return gamma0 * (T ** 5)
def Xn_eq(T, Q=Q_default):    """평형 중성자 분율"""    
    return 1.0 / (1.0 + math.exp(Q / T))# 3차시 기본 핵종 (6개)SPECIES_BASE = {    'n':   {'A': 1, 'Z': 0},   # 중성자    'p':   {'A': 1, 'Z': 1},   # 양성자    'D':   {'A': 2, 'Z': 1},   # 중수소    'T':   {'A': 3, 'Z': 1},   # 삼중수소    'He3': {'A': 3, 'Z': 2},   # 헬륨-3    'He4': {'A': 4, 'Z': 2},   # 헬륨-4}# 3차시 8반응REACTIONS_8 = [    {'name': 'p(n,g)D', 'reactants': {'p': 1, 'n': 1}, 'products': {'D': 1}, 'rate': 1.0},    {'name': 'D(p,g)He3', 'reactants': {'D': 1, 'p': 1}, 'products': {'He3': 1}, 'rate': 0.5},    {'name': 'D(n,g)T', 'reactants': {'D': 1, 'n': 1}, 'products': {'T': 1}, 'rate': 0.5},    {'name': 'D(D,n)He3', 'reactants': {'D': 2}, 'products': {'He3': 1, 'n': 1}, 'rate': 0.3},    {'name': 'D(D,p)T', 'reactants': {'D': 2}, 'products': {'T': 1, 'p': 1}, 'rate': 0.3},    {'name': 'T(D,n)He4', 'reactants': {'T': 1, 'D': 1}, 'products': {'He4': 1, 'n': 1}, 'rate': 2.0},    {'name': 'He3(D,p)He4', 'reactants': {'He3': 1, 'D': 1}, 'products': {'He4': 1, 'p': 1}, 'rate': 2.0},    {'name': 'He3(n,p)T', 'reactants': {'He3': 1, 'n': 1}, 'products': {'T': 1, 'p': 1}, 'rate': 1.0},]
def check_conservation(reaction, species):    """반응의 A, Z 보존을 검사한다."""    A_left = sum(species[s]['A'] * n for s, n in reaction['reactants'].items())    A_right = sum(species[s]['A'] * n for s, n in reaction['products'].items())    Z_left = sum(species[s]['Z'] * n for s, n in reaction['reactants'].items())    Z_right = sum(species[s]['Z'] * n for s, n in reaction['products'].items())        
print(f"A: {A_left} -> {A_right} {'OK' if A_left == A_right else 'X'}")    
print(f"Z: {Z_left} -> {Z_right} {'OK' if Z_left == Z_right else 'X'}")        
    return A_left == A_right and Z_left == Z_right
print("1-3차시 핵심 함수 로드 완료!")

---# Part J: 개념 이해 — 질량수 7과 리튬-7 문제---## J0. 3차시 복습 (Warm-up)3차시에서 배운 핵심 개념을 복습합니다.

### J0.1 He-4가 지배적인 이유He-4가 BBN의 주요 생성물인 이유로 가장 **완전한** 설명은?- (A) He-4가 가장 무겁기 때문- (B) **네트워크 합류, 안정성, 중성자가 제한 반응물**이기 때문- (C) He-4만 만들어지기 때문

In [ ]:
# J0.1: He-4가 지배적인 이유ans_J0_1 = ...  # "A", "B", "C" 중 선택

### J0.2 3차시 핵종 수3차시에서 다룬 핵종은 몇 개였는가?**힌트:** n, p, D, T, He-3, He-4

In [ ]:
# J0.2: 3차시 핵종 수ans_J0_2 = ...  # 숫자 입력

### J0.3 3차시 반응 수3차시에서 다룬 반응은 몇 개였는가?

In [ ]:
# J0.3: 3차시 반응 수ans_J0_3 = ...  # 숫자 입력

In [ ]:
grader.check('qJ0', global_env=globals())

---## J1. 질량수 7 핵종 소개### 새로운 핵종 2개| 핵종 | A | Z | 이름 | 안정성 ||------|---|---|------|--------|| $^7$Li | 7 | 3 | 리튬-7 | **안정** || $^7$Be | 7 | 4 | 베릴륨-7 | 불안정 (나중에 Li-7으로 변환) |**참고:** BBN 시기에는 Be-7도 안정처럼 행동합니다.

### J1.1 Li-7의 원자번호$^7$Li의 원자번호 Z는?**힌트:** Li는 주기율표에서 3번째 원소

In [ ]:
# J1.1: Li-7의 Zans_J1_1 = ...  # 숫자 입력

### J1.2 Be-7의 원자번호$^7$Be의 원자번호 Z는?**힌트:** Be는 주기율표에서 4번째 원소

In [ ]:
# J1.2: Be-7의 Zans_J1_2 = ...  # 숫자 입력

In [ ]:
grader.check('qJ1', global_env=globals())

---## J2. 새 반응의 보존 검사4차시에서 추가할 4개 반응입니다.### 반응 9: Be-7 생성$${}^3\text{He} + {}^4\text{He} \to {}^7\text{Be} + \gamma$$| | 왼쪽 | 오른쪽 ||--|------|--------|| A | 3 + 4 = 7 | 7 + 0 = 7 || Z | 2 + 2 = 4 | 4 + 0 = 4 |### 반응 12: Li-7 파괴$${}^7\text{Li} + p \to {}^4\text{He} + {}^4\text{He}$$| | 왼쪽 | 오른쪽 ||--|------|--------|| A | 7 + 1 = 8 | 4 + 4 = 8 || Z | 3 + 1 = 4 | 2 + 2 = 4 |

### J2.1 반응 9 보존반응 9 (${}^3\text{He} + {}^4\text{He} \to {}^7\text{Be}$)는 A와 Z를 보존하는가?

In [ ]:
# J2.1: 반응 9 보존 검사ans_J2_1 = ...  # True 또는 False

### J2.2 반응 12 보존반응 12 (${}^7\text{Li} + p \to 2\cdot{}^4\text{He}$)는 A와 Z를 보존하는가?

In [ ]:
# J2.2: 반응 12 보존 검사ans_J2_2 = ...  # True 또는 False

In [ ]:
grader.check('qJ2', global_env=globals())

---## J3. 리튬-7 문제 소개### BBN의 성공과 실패| 핵종 | BBN 예측 | 관측 | 일치? ||------|---------|------|-------|| He-4 (Yp) | ~0.247 | ~0.245 | 훌륭함 || D/H | ~2.5e-5 | ~2.5e-5 | 훌륭함 || **Li-7/H** | **~5e-10** | **~1.5e-10** | **~3배 불일치!** |> **리튬-7 문제(Lithium-7 Problem):**> 표준 BBN 이론은 관측보다 **약 3배 많은** Li-7을 예측한다.

### J3.1 리튬-7 문제리튬-7 문제의 핵심은 무엇인가?- (A) 예측이 관측보다 약 10배 크다- (B) 예측이 관측보다 약 **3배** 크다- (C) 예측과 관측이 완벽히 일치한다

In [ ]:
# J3.1: 리튬-7 문제ans_J3_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qJ3', global_env=globals())

---## J4. 불일치의 원인 가설관측과 이론이 어긋날 때, 가능한 원인:1. **관측 오차:** 측정값 자체의 불확실성2. **천체물리:** 별 내부에서 Li가 추가 파괴3. **핵물리:** 반응률 측정 오차4. **새 물리:** 알려지지 않은 새로운 현상

### J4.1 OX 문제"관측과 이론의 불일치가 있으면, 이론이 즉시 틀렸다고 결론내려야 한다."- True / False?

In [ ]:
# J4.1: 불일치 해석ans_J4_1 = ...  # True 또는 False

In [ ]:
grader.check('qJ4', global_env=globals())

---## J5. 과학적 태도> "과학은 일치하는 것만 모아 진리라고 부르지 않는다.> 오히려 어긋나는 지점이 다음 연구를 만든다."> "불일치 하나가 곧바로 전체 이론을 무너뜨리는 것은 아니다.> 성공과 실패를 동시에 다루는 태도가 과학의 성숙이다."

### J5.1 불일치의 가능한 원인리튬-7 불일치의 가능한 원인으로 적절한 조합은?- (A) 관측 오차만- (B) 이론 오차만- (C) **관측/천체물리/핵물리/새 물리** 가설이 모두 후보가 될 수 있음

In [ ]:
# J5.1: 가능한 원인ans_J5_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qJ5', global_env=globals())

---# Part K: 미니 코딩 — 4개 반응 추가---## K0. 핵종 사전 확장기존 6개 핵종에 Li-7과 Be-7을 추가합니다.

In [ ]:
# K0: 확장 핵종 사전# 기존 6개 + 새로운 2개 = 8개SPECIES_FULL = {    # 기존 핵종    'n':   {'A': 1, 'Z': 0},    'p':   {'A': 1, 'Z': 1},    'D':   {'A': 2, 'Z': 1},    'T':   {'A': 3, 'Z': 1},    'He3': {'A': 3, 'Z': 2},    'He4': {'A': 4, 'Z': 2},    # 빈칸: 새 핵종 추가    'Li7': {'A': ..., 'Z': ...},   # 힌트: A=7, Z=3    'Be7': {'A': ..., 'Z': ...},   # 힌트: A=7, Z=4}# 테스트 출력
print("SPECIES_FULL 정의 완료:")for name, props in SPECIES_FULL.items():    
print(f"  {name}: A={props['A']}, Z={props['Z']}")

In [ ]:
grader.check('qK0', global_env=globals())

---## K1. 반응 9: Be-7 생성$${}^3\text{He} + {}^4\text{He} \to {}^7\text{Be} + \gamma$$

In [ ]:
# K1: 반응 9 정의# 빈칸을 완성하세요reaction_9 = {    'reactants': {'He3': ..., 'He4': ...},  # 힌트: 각각 1개    'products': {'Be7': ...}                 # 힌트: Be7이 1개}# 보존 검사
print("반응 9: He3 + He4 -> Be7 + gamma")check_conservation(reaction_9, SPECIES_FULL)

In [ ]:
grader.check('qK1', global_env=globals())

---## K2. 반응 10: Li-7 생성$$T + {}^4\text{He} \to {}^7\text{Li} + \gamma$$

In [ ]:
# K2: 반응 10 정의# 빈칸을 완성하세요reaction_10 = {    'reactants': {'T': ..., 'He4': ...},  # 힌트: 각각 1개    'products': {'Li7': ...}               # 힌트: Li7이 1개}# 보존 검사
print("반응 10: T + He4 -> Li7 + gamma")check_conservation(reaction_10, SPECIES_FULL)

In [ ]:
grader.check('qK2', global_env=globals())

---## K3. 반응 11, 12: 변환과 파괴### 반응 11: Be-7 → Li-7 변환$${}^7\text{Be} + n \to {}^7\text{Li} + p$$### 반응 12: Li-7 파괴$${}^7\text{Li} + p \to {}^4\text{He} + {}^4\text{He}$$

In [ ]:
# K3: 반응 11, 12 정의# 빈칸을 완성하세요# 반응 11: Be7 + n -> Li7 + preaction_11 = {    'reactants': {'Be7': ..., 'n': ...},   # 힌트: 각각 1개    'products': {'Li7': ..., 'p': ...}     # 힌트: 각각 1개}# 반응 12: Li7 + p -> 2 He4reaction_12 = {    'reactants': {'Li7': ..., 'p': ...},   # 힌트: 각각 1개    'products': {'He4': ...}                # 힌트: He4가 2개!}# 보존 검사
print("반응 11: Be7 + n -> Li7 + p")check_conservation(reaction_11, SPECIES_FULL)
print()
print("반응 12: Li7 + p -> 2 He4")check_conservation(reaction_12, SPECIES_FULL)

In [ ]:
grader.check('qK3', global_env=globals())

---## K4. 보존 검사 확인

In [ ]:
# K4: 보존 검사 결과# 빈칸을 완성하세요# 반응 9의 보존 결과ans_K4_1 = ...  # 힌트: check_conservation 결과 (True/False)# 반응 12의 보존 결과ans_K4_2 = ...  # 힌트: check_conservation 결과 (True/False)
print(f"반응 9 보존: {ans_K4_1}")
print(f"반응 12 보존: {ans_K4_2}")

In [ ]:
grader.check('qK4', global_env=globals())

---

# Part L: 네트워크 시뮬레이션과 토론

---

## L0. 전체 12반응 네트워크

8개 기존 반응 + 4개 새 반응 = 12개 반응

> 🔴 **블랙박스:** 아래 코드의 `rate` 값들 (0.1, 0.05, 1.0, 0.5 등)은 **토이 모델의 임의 설정값**이다. 실제 반응률은 온도 함수이며, 핵물리 실험과 이론으로 결정된다. 특히 반응 9, 10의 반응률에는 상당한 불확실성이 있어, Li-7 문제의 원인 후보 중 하나다.

In [ ]:
# === 전체 12반응 정의 ===REACTIONS_12 = REACTIONS_8.copy()# 4개 새 반응 추가REACTIONS_12.extend([    # 9. He3 + He4 -> Be7    {'name': 'He3(He4,g)Be7', 'reactants': {'He3': 1, 'He4': 1}, 'products': {'Be7': 1}, 'rate': 0.1},    # 10. T + He4 -> Li7    {'name': 'T(He4,g)Li7', 'reactants': {'T': 1, 'He4': 1}, 'products': {'Li7': 1}, 'rate': 0.05},    # 11. Be7 + n -> Li7 + p    {'name': 'Be7(n,p)Li7', 'reactants': {'Be7': 1, 'n': 1}, 'products': {'Li7': 1, 'p': 1}, 'rate': 1.0},    # 12. Li7 + p -> 2 He4    {'name': 'Li7(p,a)He4', 'reactants': {'Li7': 1, 'p': 1}, 'products': {'He4': 2}, 'rate': 0.5},])
print(f"총 {len(REACTIONS_12)}개 반응 정의됨")for i, r in enumerate(REACTIONS_12, 1):    
print(f"  {i}. {r['name']}")

In [ ]:
# === 토이 네트워크 시뮬레이션 (확장) ===
def run_network_simulation_12(reactions=REACTIONS_12, n_steps=500, dt=0.01):    """    12반응 토이 네트워크 시뮬레이션.    """    # 초기 조건 (동결 직후, 병목이 열린 상태)    Y = {        'n': 0.12,   # 동결된 중성자 비율        'p': 0.88,   # 양성자 비율        'D': 0.0,        'T': 0.0,        'He3': 0.0,        'He4': 0.0,        'Li7': 0.0,        'Be7': 0.0,    }        history = {sp: [Y[sp]] 
for sp in Y}    time = [0]        
for step in range(n_steps):        dY = {sp: 0.0 
for sp in Y}                
for rxn in reactions:            # 반응률 계산 (단순화)            rate = rxn['rate'] * dt            for sp, count in rxn['reactants'].items():                rate *= max(0, Y[sp]) ** count                        # 반응물 소비, 생성물 생성            for sp, count in rxn['reactants'].items():                dY[sp] -= rate * count            for sp, count in rxn['products'].items():                dY[sp] += rate * count                # 업데이트        
for sp in Y:            Y[sp] = max(0, Y[sp] + dY[sp])            history[sp].append(Y[sp])                time.append((step + 1) * dt)        
    return {'time': np.array(time), 'abundances': {sp: np.array(h) for sp, h in history.items()}}# 시뮬레이션 실행result_12 = run_network_simulation_12()
print("시뮬레이션 완료!")
print(f"\n최종 풍부도:")for sp, vals in result_12['abundances'].items():    
print(f"  {sp}: {vals[-1]:.2e}")

In [ ]:
grader.check('qL0', global_env=globals())

---## L1. 그래프 해석

In [ ]:
# 그래프 그리기fig, axes = plt.subplots(1, 3, figsize=(16, 5))# 그래프 1: 모든 핵종 (로그 스케일)ax1 = axes[0]
for sp in ['n', 'p', 'D', 'T', 'He3', 'He4', 'Li7', 'Be7']:    ax1.semilogy(result_12['time'], result_12['abundances'][sp] + 1e-15, label=sp, linewidth=2)ax1.set_xlabel('Time (arbitrary)')ax1.set_ylabel('Abundance Y')ax1.set_title('All species (log scale)')ax1.legend(ncol=2)ax1.grid(True, alpha=0.3)ax1.set_ylim(1e-12, 1)# 그래프 2: 주요 핵종ax2 = axes[1]
for sp in ['n', 'He4']:    ax2.plot(result_12['time'], result_12['abundances'][sp], label=sp, linewidth=2)ax2.set_xlabel('Time (arbitrary)')ax2.set_ylabel('Abundance Y')ax2.set_title('Major species: n, He-4')ax2.legend()ax2.grid(True, alpha=0.3)# 그래프 3: 질량수 7 핵종ax3 = axes[2]ax3.plot(result_12['time'], result_12['abundances']['Li7'], 'r-', label='Li-7', linewidth=2)ax3.plot(result_12['time'], result_12['abundances']['Be7'], 'b--', label='Be-7', linewidth=2)ax3.set_xlabel('Time (arbitrary)')ax3.set_ylabel('Abundance Y')ax3.set_title('A=7 species: Li-7, Be-7')ax3.legend()ax3.grid(True, alpha=0.3)plt.tight_layout()plt.show()# 수치 출력
print(f"\n최종 Li-7/H: {result_12['abundances']['Li7'][-1]:.2e}")
print(f"최종 Be-7/H: {result_12['abundances']['Be7'][-1]:.2e}")

### L1.1 Li-7 풍부도시뮬레이션에서 Li-7의 최종 풍부도는 다른 핵종에 비해 어떠한가?- (A) He-4보다 크다- (B) 매우 **작다** (10^-4 이하)- (C) n과 비슷하다

In [ ]:
# L1.1: Li-7 풍부도ans_L1_1 = ...  # "A", "B", "C" 중 선택

### L1.2 Be-7의 변화시간에 따른 Be-7의 변화는?- (A) **올라갔다가 내려간다** (중간물질 패턴)- (B) 계속 증가한다- (C) 변화 없다

In [ ]:
# L1.2: Be-7 변화ans_L1_2 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qL1', global_env=globals())

---## L2. 민감도 실험반응 12의 반응률을 바꾸면 Li-7 결과가 어떻게 변하는지 관찰합니다.

In [ ]:
# 민감도 실험: 반응 12 (Li7 파괴) 반응률 변화# 기본 (x1)result_base = run_network_simulation_12()# 반응 12 반응률 2배reactions_x2 = [r.copy() 
for r in REACTIONS_12]reactions_x2[11]['rate'] = 1.0  # 0.5 -> 1.0result_x2 = run_network_simulation_12(reactions=reactions_x2)# 반응 12 반응률 0.5배reactions_x05 = [r.copy() 
for r in REACTIONS_12]reactions_x05[11]['rate'] = 0.25  # 0.5 -> 0.25result_x05 = run_network_simulation_12(reactions=reactions_x05)# 비교 그래프plt.figure(figsize=(10, 5))plt.plot(result_base['time'], result_base['abundances']['Li7'], 'b-', linewidth=2, label='Base (x1)')plt.plot(result_x2['time'], result_x2['abundances']['Li7'], 'r--', linewidth=2, label='Li7(p,a)He4 x2')plt.plot(result_x05['time'], result_x05['abundances']['Li7'], 'g:', linewidth=2, label='Li7(p,a)He4 x0.5')plt.xlabel('Time (arbitrary)')plt.ylabel('Li-7 Abundance')plt.title('Sensitivity: Li-7 destruction rate')plt.legend()plt.grid(True, alpha=0.3)plt.show()
print(f"Base Li-7: {result_base['abundances']['Li7'][-1]:.2e}")
print(f"x2 Li-7:   {result_x2['abundances']['Li7'][-1]:.2e} (change: {(result_x2['abundances']['Li7'][-1]/result_base['abundances']['Li7'][-1]-1)*100:+.0f}%)")
print(f"x0.5 Li-7: {result_x05['abundances']['Li7'][-1]:.2e} (change: {(result_x05['abundances']['Li7'][-1]/result_base['abundances']['Li7'][-1]-1)*100:+.0f}%)")

### L2.1 민감도 해석반응 12의 반응률을 바꾸면 Li-7 결과가 변하는 것을 관찰했습니다. 이것은 무엇을 의미하는가?- (A) 반응률은 결과에 영향을 주지 않는다- (B) Li-7 예측이 특정 반응의 **반응률에 민감**하다- (C) 보존법칙이 깨진다

In [ ]:
# L2.1: 민감도 해석ans_L2_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qL2', global_env=globals())

---## L3. 과학철학 토론### 토론 질문1. 불일치의 원인을 어디부터 의심해야 하는가?2. "모형의 성공(He, D)과 실패(Li)"를 동시에 받아들이는 태도는 무엇인가?

### L3.1 과학적 점검 순서관측과 이론이 어긋날 때, 과학은 어떤 순서로 원인을 점검해야 하는가?- (A) 새로운 물리부터 고려한다- (B) 이론 전체를 버린다- (C) **간단한 설명(관측 오차, 기존 물리)부터 점검**하고, 새 물리는 마지막에 고려한다

In [ ]:
# L3.1: 과학적 순서ans_L3_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qL3', global_env=globals())

---## L4. Exit Ticket — 최종 이해도 점검

### L4.1 리튬-7 문제 요약"리튬-7 문제"를 가장 잘 요약한 것은?- (A) Li-7이 전혀 만들어지지 않는다- (B) 표준 BBN **예측이 관측보다 약 3배 높다**- (C) Li-7과 Be-7이 같은 양으로 만들어진다

In [ ]:
# L4.1: 리튬-7 문제 요약ans_L4_1 = ...  # "A", "B", "C" 중 선택

### L4.2 OX 문제"Li-7 불일치 때문에 BBN은 실패한 이론이다."- True / False?

In [ ]:
# L4.2: BBN 평가ans_L4_2 = ...  # True 또는 False

### L4.3 심화 학습 방향4차시를 마친 후, 더 심화 학습을 위한 방향으로 적절한 것은?- (A) 정밀 BBN 코드(PRIMAT, AlterBBN) 살펴보기- (B) BBN을 포기하고 다른 이론 찾기- (C) Li-7 문제가 해결되었다고 결론 내리기

In [ ]:
# L4.3: 심화 학습ans_L4_3 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qL4', global_env=globals())

---

## ⚠️ 오해하기 쉬운 개념

### 오해 1: "불일치 = 이론이 틀렸다"

**정정:** 불일치는 **어딘가에 빠진 것이 있다**는 신호다.
D와 He-4에서는 놀라운 일치를 보이므로, 이론 전체가 틀린 것이 아니라 특정 부분에 개선이 필요할 수 있다는 뜻이다.

### 오해 2: "Li-7 문제는 해결되었다"

**정정:** 2024년 현재에도 Li-7 문제는 **미해결**이다.
물론 여러 가설(관측 오차, 천체물리, 핵물리, 새 물리)이 제안되었지만, 결정적인 해결책은 아직 없다.
바로 이것이 활발한 연구 주제인 이유다.

### 오해 3: "BBN은 실패한 이론이다"

**정정:** BBN은 D, He-3, He-4에서 **놀라운 성공**을 거두었다.
Li-7 불일치는 전체 이론의 실패가 아니라, **정밀한 테스트**의 결과로 보아야 한다.
성공과 실패를 동시에 인정하는 것이 과학적 태도다.

---# 4차시 핵심 정리## 세 가지 큰 질문과 답### Q1. 질량수 7은 어떤 경로로 만들어지는가?**A:** 두 가지 주요 경로가 있다:1. **He-3 경로:** He-3 + He-4 -> Be-7 -> Li-72. **T 경로:** T + He-4 -> Li-7실제로는 **Be-7 경로**가 지배적이다.### Q2. 표준 BBN은 D와 He에서는 잘 맞는데, 왜 Li-7에서 긴장이 남는가?**A:** 여러 가능성이 있다:1. 관측 오차 (별 대기 측정의 불확실성)2. 천체물리 효과 (별 내부에서 Li 파괴)3. 핵물리 오차 (반응률 측정 불확실성)4. 새로운 물리 (미지의 현상)**현재 미해결 문제**로 남아 있다.### Q3. 관측과 이론이 어긋날 때, 과학은 어떤 순서로 원인을 점검해야 하는가?**A:**1. 관측 오차 점검2. 기존 물리 효과 재검토3. 이론의 가정 검토4. 새로운 물리 고려 (마지막 수단)> **원칙:** 간단한 설명부터 점검하고, 복잡한 설명은 마지막에 고려한다.---## 프로젝트 전체 요약| 차시 | 핵심 개념 | 핵심 질문 ||------|----------|----------|| 1차시 | 동결, Gamma/H, Yp=2Xn | 왜 평형이 깨지는가? || 2차시 | 중수소 병목, 스위치 | 왜 D가 바로 쌓이지 않는가? || 3차시 | 보존법칙, 8반응 | 어떻게 He-4로 가는가? || **4차시** | **A=7, Li-7 문제** | **왜 Li-7만 안 맞는가?** |---## 과학적 메시지> "과학은 일치하는 것만 모아 진리라고 부르지 않는다.> 오히려 어긋나는 지점이 다음 연구를 만든다."> "불일치 하나가 곧바로 전체 이론을 무너뜨리는 것은 아니다.> 성공과 실패를 동시에 다루는 태도가 과학의 성숙이다."---축하합니다! 전체 프로젝트를 완료했습니다!

In [ ]:
# 전체 채점grader.check_all()